In [5]:
from sage.sat.solvers.dimacs import DIMACS
from sage.all import *
from sage.combinat.posets.poset_examples import Posets
import itertools

In [6]:
from enum import Enum

class QT(Enum):
    Less = 1
    YesString = 2

In [7]:
def ConstructVmap(pset, ldim):
    lm = {}
    count = 1
    for a,b in itertools.combinations(pset, int(2)):
        for i in range(ldim):
            lm[a,b,i] = count
            count += 1
    ym = {}
    for s in itertools.product([0,1], repeat=int(ldim)):
        ym[s] = count
        count += 1

    def vmap(qtype, *args):
        if qtype == QT.Less:
            a, b, i = args
            if b < a:
                return -lm[b,a,i]
            return lm[a,b,i]
        elif qtype == QT.YesString:
            return ym[args]
    return vmap

In [8]:
def Transitivity(pset, ldim, vn):
    cl = []
    for a, b, c in itertools.permutations(pset, int(3)):
        for i in range(ldim):
            xab = vn(QT.Less, a, b, i)
            xbc = vn(QT.Less, b, c, i)
            xac = vn(QT.Less, a, c, i)
            cl.append((-xab, -xbc, xac))
    return cl

In [11]:
def OtherThanDim(ldim, vn):
    return [
        tuple(vn(QT.YesString, *x) for x in itertools.product([0,1], repeat=int(ldim)) if x != tuple(1 for _ in range(ldim))),
         (vn(QT.YesString, *tuple(1 for _ in range(ldim))),)
    ]

In [13]:
OtherThanDim(3, ConstructVmap([1,2,3], 3))

[(10, 11, 12, 13, 14, 15, 16), (17,)]

In [14]:
def PosetClauses(dG, ldim):
    vn = ConstructVmap(sorted(dG.vertices()), ldim)
    cl = []
    for a, b in sorted(dG.edges(labels=False)):
        for mask in itertools.product([0, 1], repeat=int(ldim)):
            c = [ -vn(QT.Less, a, b, i) if mask[i] == 1 else -vn(QT.Less, b, a, i) for i in range(ldim)] + [vn(QT.YesString, *mask)]
            cl.append(tuple(c))
    for a, b in sorted(dG.complement().edges(labels=False)):
        for mask in itertools.product([0, 1], repeat=int(ldim)):
            c = [ -vn(QT.Less, a, b, i) if mask[i] == 1 else -vn(QT.Less, b, a, i) for i in range(ldim)] + [-vn(QT.YesString, *mask)]
            cl.append(tuple(c))
    return cl, vn

In [15]:
def BooleanGraph(dim):
    B = Posets.BooleanLattice(dim)
    return DiGraph([x for x in B.relations() if x[0] != x[1]])

In [26]:
def WriteProblem(dim, ldim, fname):
    dG = BooleanGraph(dim)
    clauses, vn = PosetClauses(dG, ldim)
    clauses += OtherThanDim(ldim, vn)
    clauses += Transitivity(dG.vertices(), ldim, vn)
    s = DIMACS()
    for c in clauses:
        s.add_clause(c)
    s.clauses(fname)

In [17]:
from functools import cmp_to_key

def RecoverOrder(val, vn, dim, lin_ext_idx):
    ordering = [e for e in range(1 << dim)]

    def Compare(a, b):
        if val[vn(QT.Less, a, b, lin_ext_idx)]:
            return -1
        else:
            return 1

    return sorted(ordering, key=cmp_to_key(Compare))

In [18]:
def PrintMasks(val, vn, ldim):
    for mask in itertools.product([0,1], repeat=int(ldim)):
        print(f"{mask} -> {'YES' if val[vn(QT.YesString, *mask)] else 'NO'}")

In [48]:
def ToList(data):
    return data.strip().split(' ')

In [49]:
def ToMap(xd):
    val = {i:int(xd[i]) > 0 for i in range(len(xd))}
    val.update({-i : not (int(xd[i]) > 0) for i in range(len(xd))})
    return val

In [52]:
data = """-1 -2 -3 4 -5 -6 -7 8 -9 -10 -11 12 -13 -14 -15 16 -17 -18 -19 20 -21 -22 -23 24 -25 -26 -27 28 -29 -30 -31 32 -33 -34 -35 36 -37 -38 -39 40 -41 -42 -43 44 -45 -46 -47 48 -49 -50 -51 52 -53 -54 -55 56 -57 -58 -59 60 -61 62 -63 -64 -65 66 -67 68 -69 70 -71 -72 -73 74 -75 76 -77 78 -79 -80 -81 82 -83 84 -85"""

In [53]:
result_list = data.strip().split(' ')

In [55]:
values = {i:int(result_list[i]) > 0 for i in range(len(result_list))}
values.update({-i : not (int(result_list[i]) > 0) for i in range(len(result_list))})
N = 3
vn = ConstructVmap(BooleanGraph(N).vertices(), N)
ords = [RecoverOrder(values, vn, N, j) for j in range(N)]
ords

[[3, 6, 2, 4, 7, 1, 5, 0], [4, 3, 7, 5, 1, 6, 2, 0], [4, 5, 7, 3, 2, 6, 0, 1]]

In [28]:
WriteProblem(3, 3, "test.dimacs")